In [70]:
import ast
import pandas as pd

games = []
with open('steam_games.json') as f: #permite acceder al contenido del archivo JSON
    for line in f.readlines():
        games.append(ast.literal_eval(line)) #convirtir cada línea del archivo JSON en un diccionario de Python y añadiéndolo a la lista games

df = pd.DataFrame(games) # carga los datos en un DataFrame de pandas

In [71]:
# Ver los primeros registros del DataFrame
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [72]:
#revisar las columnas y tipos de datos presentes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [73]:
# Cambiar el dtype de la columna'release_date' ya que contiene fechas en formato de texto
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [74]:
#Cambiar el dtype de acolumna llamada 'metascore' ya que contiene puntajes en formato de texto
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

In [75]:
from fastapi import FastAPI 

In [76]:
# Crear una instancia de FastAPI
app = FastAPI()

In [101]:
# Endpoint 1: Géneros más vendidos por año
@app.get('/ Genero')
def genero(año: str):
    #Filtrar los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]
    #Agrupar los géneros y cuenta la cantidad de juegos en cada género
    generos_mas_vendidos = df_filtrado['genres'].explode().value_counts().nlargest(5).index.to_list()

    return generos_mas_vendidos

In [78]:
# Endpoint 2: Juegos lanzados en un año
@app.get('/ Juegos')
def juegos(año: str):
    # Filtra los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]

    # Obtiene la lista de juegos lanzados en el año
    juegos_lanzados = df_filtrado['title'].todict()

    return juegos_lanzados

In [79]:
# Endpoint 3: Specs más comunes por año
@app.get('/ Specs')
def specs(año: str):
    #Filtrar los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]

    #Concatenar las listas de Specs en una sola lista
    specs_list = df_filtrado['specs'].explode().tolist()

    #Contar la frecuencia de cada Spec
    specs_count = pd.Series(specs_list).value_counts()

    #Se obtienen las 5 Specs más comunes
    specs_mas_comunes = specs_count.nlargest(5).index.todict()

    return specs_mas_comunes

In [80]:
# Endpoint 4: Cantidad de juegos con early access en un año
@app.get('/ Earlyacces')
def earlyacces(año: str):
    #Filtrar los datos por el año especificado y por juegos con early access
    df_filtrado = df[(df['release_date'].dt.year == int(año)) & (df['early_access'] == True)]

    #Contar la cantidad de juegos con early access en el año especificado
    cantidad_juegos_early_access = len(df_filtrado)

    return cantidad_juegos_early_access

In [81]:
# Endpoint 5: Análisis de sentimiento por año
@app.get('/ Sentimient')
def sentiment(año: str):
    #Filtrar los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]

    #Contar la cantidad de registros que cumplen con cada análisis de sentimiento
    sentimiento_counts = df_filtrado['sentiment'].value_counts()

    #Convertir la serie de conteo en un diccionario
    sentimiento_dict = sentimiento_counts.to_dict()

    return sentimiento_dict

In [92]:
# Endpoint 6: Top 5 juegos con mayor metascore por año
@app.get('/ Metascore')
def metascore(año: str):
    #Filtrar los datos por el año especificado y ordena por metascore de forma descendente
    df_filtrado = df[df['release_date'].dt.year == int(año)].sort_values(by='metascore', ascending=False)

    #Seleccionar los 5 juegos con mayor metascore y obtiene su información
    top_juegos_metascore = df_filtrado.head(5)[['title', 'metascore']].to_dict(orient='records')

    return top_juegos_metascore